## MATH50003 Week 1 Notes 

In [3]:
using SetRounding, ColorBitstring

In [21]:
#8-bit unsigned integers
x = UInt8(255)
y = UInt8(1)
println(x); printbits(x); println("+"); printbits(y); println("=")
printbits(x+y)


"""
print() and println() do the same thing except println() will add an new line to the end of the output. 
printbits() will output the bit representation of the number.
"""

255
11111111+
00000001=
00000000

"print() and println() do the same thing except println() will add an new line to the end of the output. \nprintbits() will output the bit representation of the number.\n"

In [22]:
#8-bit signed integers
println(Int8(55)); printlnbits(-Int8(55))

55
11001001


In [24]:
#Find the largest and smallest instances of 8-bit integers
printlnbits(typemax(Int8)) #2^7 - 1 = 127
printlnbits(typemin(Int8)) #- 2^7 = -128

01111111
10000000


In [36]:
#Division
println(5 ÷ 2 == div(5,2)) 
println(5 ÷ 2) # Give integer
println(5 / 2 == 2 \ 5) # \ means division on the right
println(5 / 2) # / and \ give floating point number
println(5 // 2) # // gives rational number


true
2
true
2.5
5//2


In [48]:
#Float32 is represented by f0 in Julia
printlnbits(1f0/3) #Float32 of 1/3

#Default is Float64
printlnbits(-1/3)

00111110101010101010101010101011
1011111111010101010101010101010101010101010101010101010101010101


In [53]:
#Find the machine epsilon / smallest positive normal number/ largest normal number of given type of instance Float32
println(eps(Float32))
println(floatmin(Float32))
println(floatmax(Float32))
printlnbits(floatmin(Float32))
printlnbits(floatmin(Float32)/2) #Divide the smallest normal number by 2 giving the subnormal number

1.1920929e-7
1.1754944e-38
3.4028235e38
00000000100000000000000000000000
00000000010000000000000000000000


### Not a number: $F^{special}:=\{\infty, -\infty, NaN \}$


Whenever the exponent bits are all 1, then they represent an element of $F^{special}$.

If all the significant bits are all 0, the number represents either $\pm \infty$.
If not, the number represent NaN.





In [56]:
#Example of 16-bit 
#for 64-bit, it is just Inf / NaN
printlnbits(Inf16)
printlnbits(-Inf16)
printlnbits(NaN16)

0111110000000000
1111110000000000
0111111000000000


In [60]:
#Rounding
printlnbits(1/3) #64 bits
printlnbits(Float32(1/3)) #By default, round to nearest 32-bit
printlnbits(Float32(1/3, RoundDown)) #Round down 
printlnbits(Float32(1/3, RoundUp)) #Round up

0011111111010101010101010101010101010101010101010101010101010101
00111110101010101010101010101011
00111110101010101010101010101010
00111110101010101010101010101011


In [62]:
#Use setrounding to change the rounding mode for a chunk of code
#Compute upper and lower bounds for /

x = 1f0 # 1 in Float32

setrounding(Float32, RoundDown) do 
    x/3
end, #Use comma to output a tuple
setrounding(Float32, RoundUp) do 
    x/3
end

(0.3333333f0, 0.33333334f0)

### Rounding arithmetic

If x $\in \mathbb{N}$, then $fl^{mode}(x)=x(1+\delta_x^{mode}$) where the relative error is $|\delta_x^{nearest}| \leq \frac{\epsilon_m}{2}$ or $|\delta_x^{up/down}| \leq \epsilon_m$

In [64]:
abs(3.6 .- (1.1 .+ 1.2 .+ 1.3)), 4.8eps()

(4.440892098500626e-16, 1.0658141036401502e-15)

In [65]:
printlnbits(Int8(10))

00001010


### Question
Let $m(y)=min\{x\in F_{32}:x > y\}$ be the smallest single precision number than y. 

Find m(2) and m(2)-2

In [76]:
#Find the smallest single precision (Float32) nunmber than 2
println(nextfloat(2f0))

#m(2) - 2
nextfloat(2f0) - 2

2.0000002


2.3841858f-7

### Question
The following problems consider implementation of interval arithmetic for
proving precise bounds on arithmetic operations. That is recall the set operations
$$
A + B = \{x + y : x \in A, y \in B\}, AB = \{xy : x \in A, y \in B\}.
$$

First,

For intervals $A = [a,b]$ and $B = [c,d]$ such that $0 \notin A,B$
 and integer $n \neq 0$, 
deduce formulas for the minimum and maximum of $A/n$, $A+B$ and $AB$.

Secondly,

We want to implement floating point variants such that, for $S = [a,b] + [c,d]$
 $P = [a,b] * [c,d]$, and $D = [a,b]/n$ for an integer $n$,
$$
\begin{align*}
[a,b] ⊕ [c,d] &:= [{\rm fl}^{\rm down}(\min S), {\rm fl}^{\rm up}(\max S)] \\
[a,b] ⊗ [c,d] &:= [{\rm fl}^{\rm down}(\min P), {\rm fl}^{\rm up}(\max P)] \\
[a,b] ⊘ n &:= [{\rm fl}^{\rm down}(\min D), {\rm fl}^{\rm up}(\max D)]
\end{align*}
$$
This guarantees $S ⊆ [a,b] ⊕ [c,d]$, $P ⊆ [a,b] ⊗ [c,d]$, and
$D ⊆ [a,b] ⊘ n$.
In other words, if $x \in [a,b]$ and
$y \in [c,d]$ then $x +y \in [a,b] ⊕ [c,d]$, and we thereby have  bounds on $x + y$.

Use the formulae from Problem 5.1 to complete (by replacing the `# TODO: …` comments with code)
 the following implementation of an 
`Interval` 
so that `+`, `-`, and `/` implement $⊕$, $⊖$, and $⊘$ as defined above.

For the first part,
$$
\begin{align*}
{A \over n} &= \begin{cases}
[a/n,b/n] & n > 0 \\
[b/n,a/n] & n < 0
\end{cases},\\
A + B &= [a + c, b + d] \\
AB &= \begin{cases}
[bd,ac]& a,b,c,d < 0 \\
[ad,bc]& a,b < 0 \hbox{ and } c,d >0 \\
[bc,ad]& a,b > 0 \hbox{ and } c,d  < 0 \\
[ac,bd]& a,b,c,d > 0
\end{cases}
\end{align*}
$$

In [82]:
# Interval(a,b) represents the closed interval [a,b]
struct Interval{T}
    a::T
    b::T
end

import Base: *, +, -, /, one, in

# create an interval corresponding to [1,1]
one(x::Interval) = Interval(one(x.a), one(x.b))

# Support x in Interval(a,b)
in(x, y::Interval) = y.a ≤ x ≤ y.b

# Following should implement ⊕
function +(x::Interval, y::Interval)
    T = promote_type(typeof(x.a), typeof(x.b))
    a = setrounding(T, RoundDown) do
        # TODO: lower bound
        x.a + y.a
    end
    b = setrounding(T, RoundUp) do
        # TODO: upper bound
        x.b + y.b
    end
    Interval(a, b)
end

# Following should implement ⊘
function /(x::Interval, n::Integer)
    T = typeof(x.a)
    if iszero(n)
        error("Dividing by zero not support")
    end
    a = setrounding(T, RoundDown) do
        # TODO: lower bound
        if n > 0
            x.a / n
        else
            x.b / n
        end
    end
    b = setrounding(T, RoundUp) do
        # TODO: upper bound
        if n > 0
            x.b / n
        else
            x.a /n
        end
    end
    Interval(a, b)
end

# Following should implement ⊗
function *(x::Interval, y::Interval)
    T = promote_type(typeof(x.a), typeof(x.b))
    if 0 in x || 0 in y
        error("Multiplying with intervals containing 0 not supported.")
    end
    a = setrounding(T, RoundDown) do
        # TODO: lower bound
        if x.a < 0 && x.b < 0 && y.a < 0 && y.b < 0 # && means 'and', || means 'or'
            x.b * y.b
        elseif x.a < 0 && x.b < 0 && y.a > 0 && y.b > 0
            x.a * y.b
        elseif x.a > 0 && x.b > 0 && y.a < 0 && y.b < 0
            x.b * y.a
        else
            x.a * y.a
        end
    end
    b = setrounding(T, RoundUp) do
        # TODO: upper bound
        if x.a < 0 && x.b < 0 && y.a < 0 && y.b < 0 # && means 'and', || means 'or'
            x.a * y.a
        elseif x.a < 0 && x.b < 0 && y.a > 0 && y.b > 0
            x.b * y.a
        elseif x.a > 0 && x.b > 0 && y.a < 0 && y.b < 0
            x.a * y.b
        else
            x.b * y.b
        end
    end
    Interval(a, b)
end

* (generic function with 365 methods)